## Imports

In [2]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [14]:
df = pd.read_csv('SoFlaTopRentals_013122 - SoFla_rentals_Realtor_013122.csv')

In [15]:
df = df.head(10)
df = df.filter(items=['type','price_per_month','address','Listing agent '])

In [18]:
df.at[0,'address']=('190 Palm Ave, Miami Beach, FL 33139')

## Google Maps API Geolocater Setup

In [20]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [22]:
df['geo_address'] = df['address']

In [23]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [24]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Correction Section

In [25]:
df.columns

Index(['type', 'price_per_month', 'address', 'Listing agent ', 'geo_address',
       'geocoded', 'lat', 'lon'],
      dtype='object')

## HTML Popup Formatter

In [26]:
def popup_html(row):
    Address = row['address']
    price_per_month = row['price_per_month']
    List_Agent = row['Listing agent ']
    Type = row['type']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Price Per Month: </strong>{}'''.format(price_per_month) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(List_Agent) + '''<br>
    <strong>Description: </strong>{}'''.format(Type) + '''<br>
    </html>
    '''
    return html

In [27]:
df.columns

Index(['type', 'price_per_month', 'address', 'Listing agent ', 'geo_address',
       'geocoded', 'lat', 'lon'],
      dtype='object')

## Map Maker

In [30]:
### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)


### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Top Rental Listings in South Florida")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

# Display map
m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [17]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/manhattan_dev_map_01022023
